# DeepLabCut for markerless pose estimation

In [0]:
!pip install deeplabcut

In [0]:
%tensorflow_version 1.x

In [0]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
ProjectFolderName = 'orientation-invariant-joshua-2020-06-04'
VideoType = 'mov'

# Folder of videos or video to analyse
videofile_path = ['/content/drive/My Drive/fyp/'+ProjectFolderName+'/videos/'] #Enter the list of videos or folder to analyze.


In [0]:
import os
os.environ['DLClight']= 'True'

In [0]:
import deeplabcut

In [0]:
# Path to config file
path_config_file = '/content/drive/My Drive/fyp/'+ProjectFolderName+'/config.yaml'
path_config_file


# Create training dataset

In [0]:
import numpy as np
import math

In [0]:
def removeElementsStartingAtIndex(arr, lower, numel):
  # [0,1,2,3,4,5,6,7,8,9,10] remove lower=3, numel=3
  # returns [0,1,2,6,7,8,9,10]
  return arr[:lower] + arr[lower+numel:]

def removeElementsInArr(arr, lowerList, numel):
  '''
  Removes a specified number of elements in arr starting at corresponding elements in lowerList
  '''
  new_arr = np.array(arr,dtype='int')
  for index,lower in enumerate(lowerList):
    removeElements = [i for i in range(lower, lower+numel[index])]
    for x in removeElements:
      new_arr = np.delete(new_arr, np.argwhere(new_arr == x))
  return new_arr

def makeKFoldSplit(k, numElements):
  '''
  Make k-fold cross val split
  '''
  allinds = [i for i in range(numElements)]
  test_splits = []
  train_splits = []
  trainfrac = 1 - 1/k
  num_test = math.ceil((1 - trainfrac)*numElements)
  for i in range(k):
    start_ind = i*math.ceil(num_test)
    test_fold = [i for i in range(start_ind, start_ind+num_test)]
    train_fold = allinds[0:start_ind] + allinds[start_ind+num_test:]
    test_splits.append(test_fold)
    train_splits.append(train_fold)
  return train_splits, test_splits

In [0]:
train, test = makeKFoldSplit(5, 415)

In [0]:
allTrainShuffle1, shuffle1test = deeplabcut.mergeandsplit(path_config_file, trainindex=0, uniform=False)
shuffle1train = removeElementsInArr(allTrainShuffle1, lowerList=[0], numel=[48])[:48]
shuffle2train = removeElementsInArr(allTrainShuffle1, lowerList=[73], numel=[74])

print('Shuffle 1')
print('All train: ', allTrainShuffle1)
print('Filtered train: ', shuffle1train)
print('Test: ', shuffle1test)

print('Shuffle 2')
# print('All train: ', allTrainShuffle2)
print('Filtered train: ', shuffle2train)
# print('Test: ', shuffle3test)


In [0]:
deeplabcut.create_training_dataset(path_config_file, Shuffles=[13], trainIndexes=[train[4]], testIndexes=[test[4]], net_type='resnet_50', augmenter_type='imgaug')

# Start training



In [0]:
deeplabcut.train_network(path_config_file, shuffle=13, max_snapshots_to_keep=8, displayiters=10,saveiters=10000,maxiters=40000)

# Evaluate network



In [0]:
deeplabcut.evaluate_network(path_config_file, Shuffles=[13],plotting=False)

# Analyze videos



In [0]:
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype=VideoType)

# Label new video/s


In [0]:
deeplabcut.create_labeled_video(path_config_file,videofile_path, videotype=VideoType)